# Master TIDE - Conférences Python 2020

Francis Wolinski

&copy; 2020 Yotta Conseil

In [ ]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
%matplotlib inline

# display options
pd.set_option("display.max_rows", 20)

## Analyse en composantes principales

L'analyse en composantes principales (ACP) est une procédure statistique qui utilise une transformation orthogonale pour convertir un ensemble d'observations de variables numériques potentiellement corrélées vers un ensemble de valeurs non corréelées appelées composantes principales.

Cette procédure statistique consiste à réduire le nombre de dimensions, c'est-à-dire, à représenter un jeu de données avec un nouveau jeu de dimension moindre. Si l'on choisit de réduire à 2 dimensions, il est possible de visualiser les données sur un graphique 2D.

#### Dataset

Le jeu de données provient des Nations Unies (UNCTAD) et concerne les facteurs de développement du e-commerce par pays :
- % d'individus qui utilisent internet, source : International Telecommunication Union (ITU)
- % d'individus de 15 ans et plus qui ont un compte bancaire, source : Banque Mondiale
- nombre de serveurs internet sécurisés par million d'habitants, source : Banque Mondiale
- score de fiabilité postale, from Universal Postal Union (UPU)

Source: https://unctad.org/en/PublicationsLibrary/tn_unctad_ict4d14_en.pdf

In [ ]:
# load e-commerce index
df_ecom = pd.read_csv('e-commerce_index.txt',
                      sep=';',
                     usecols=range(1, 6))
df_ecom.shape

In [ ]:
# head
df_ecom.head()

Extract the country and the 4 columns with the e-commerce factors.

### Application de l'ACP


On extrait les 4 colonnes numériques.

In [ ]:
# extraction des 4 colonnes numériques
data = df_ecom.iloc[:, 1:6]
data.head()

On normalise les données avec `scikit-learn`. Chaque colonne est normalisée avec la formule $\frac{x - \mu}{\sigma}$. Les nouvelles colonnes ont une moyenne de 0 et un écart-type de 1.

In [ ]:
# import et application d'un StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_std = scaler.fit_transform(data)
data_std

In [ ]:
# shape
data_std.shape

In [ ]:
# la moyenne vaut 0.0
data_std.mean(axis=0)

In [ ]:
# vérification avec allclose()
np.allclose(data_std.mean(axis=0), 0.0)

In [ ]:
# l'écart-type vaut 1.0
data_std.std(axis=0)

In [ ]:
# vérification avec allclose()
np.allclose(data_std.std(axis=0), 1.0)

Application d'une ACP avec `scikit-learn`. le nombre de composantes est 2, ce qui va permettre de représenter les données sur un graphique.

In [ ]:
# import et application d'une ACP
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
result = pca.fit_transform(data_std)
result

In [ ]:
# shape
result.shape

### Affichage des résultats

In [ ]:
# affichage des résultats
pays = df_ecom['Country']
x = -result.T[0]
y = -result.T[1]

# plot the figure
fig, ax = plt.subplots(figsize=(12, 12))

# scatterplot
sns.scatterplot(x=x, y=y, ax=ax)

# axes
ax.plot(plt.xlim(), [0.0, 0.0], ls=':', lw=1, color='k')
ax.plot([0.0, 0.0], plt.ylim(), ls=':', lw=1, color='k')

# labels
offset = 0.005
for label, x, y in zip(pays, x, y):
    ax.text(x+offset, y+offset, label, fontsize=8);

### Interprétation des résultats

La variable `explained_variance_ratio_` vaut le pourcentage de la variance expliquée par chacune des composante. Les 2 facteurs expliquent 90% de la variance des données.

In [ ]:
# explained_variance_ratio_
pca.explained_variance_ratio_

In [ ]:
# somme explained_variance_ratio_
pca.explained_variance_ratio_.sum()

La variable `components_` vaut les axes principaux projetés dans l'espace des données.

In [ ]:
# components
pca.components_

In [ ]:
# on vérifie qu'ils sont orthogonaux
pca.components_[0]@pca.components_[1]  # ou pca.components_[0].dot(pca.components_[1])

In [ ]:
# vérification avec allclose()
np.allclose(pca.components_[0]@pca.components_[1], 0.0)

Par ailleurs, comme la première composante est négative dans toutes les dimensions et que la plus grande valeur absolue de la seconde composante est négative, nous allons prendre les opposés des axes X et Y afin d'avoir une représentation plus standard : en bas à gauche les petites valeurs et en haut à droite les grandes valeurs.

### Scree plot

Il est également possible d'effectuer une ACP avec l'ensemble des composantes pour déterminer empiriquement le nombre de composantes à retenir avec la "cassure" formée dans un "scree plot".

In [ ]:
# affichage de la variance expliquée en fonction des composantes
n = 4
pca = PCA(n_components=n)
result = pca.fit_transform(data_std)
eigenval = (n-1)/n * pca.explained_variance_ratio_
fig, ax = plt.subplots()
ax.plot(np.arange(n)+1, eigenval)
ax.set_title('Scree plot')
ax.set_xticks(range(1, 5))
ax.set_xlabel('Factor number')
ax.set_ylabel('Eigen values');

In [ ]:
# cumul de variance expliquée
fig, ax = plt.subplots()
ax.plot(np.arange(n)+1, np.cumsum(pca.explained_variance_ratio_))
ax.set_title("Explained variance vs. # of factors")
ax.set_xticks(range(1, 5))
ax.set_xlabel("Factor number")
ax.set_ylabel("Cumsum explained variance ratio");

### Amélioration du graphique

#### Dataset 2

Ajout des informations sur les pays : continent et population.

Source: https://www.geonames.org/countries/

In [ ]:
import os

# require internet access
if os.path.exists('GeoNames.htm'):
    var = pd.read_html('GeoNames.htm',
                       header=0,
                       encoding='latin-1',
                       keep_default_na=False)
else:
    var = pd.read_html('https://www.geonames.org/countries/',
                       header=0,
                       encoding='latin-1',
                       keep_default_na=False)
    
# display the shapes of the found tables
pays = var[1]
pays.head()

On va effectuer une jointure entre les données de e-commerce et les pays pour pouvoir utiliser les informations de continent et de population. Faute de code ISO dans le fichier e-commerce, on va effectuer une jointure sur le nom des pays.

In [ ]:
# join with the Country dataset on the common `Country` column
df_ecom2 = pd.merge(df_ecom, pays, on='Country', how='inner')
df_ecom2.shape

On remarque qu'il manque des pays...

In [ ]:
# pays absents
df_ecom.loc[~df_ecom['Country'].isin(df_ecom2['Country'].unique()), 'Country'].unique().tolist()

On effectue un mapping manuel des pays manquants.

In [ ]:
# mapping
mapping = {
    'United States of America': 'United States',
    'China, Hong Kong SAR': 'Hong Kong',
    'Korea, Republic of': 'North Korea',
    'Russian Federation': 'Russia',
    'Iran (Islamic Republic of)': 'Iran',
    'Republic of Moldova': 'Moldova',
    'Viet Nam': 'Vietnam',
    'Venezuela (Bolivarian Rep. of)': 'Venezuela',
    'United Republic of Tanzania': 'Tanzania',
    'Bolivia (Plurinational State of)': 'Bolivia',
    'Lao People\'s Dem. Rep.': 'Laos',
    'Cōte d\'Ivoire': 'Ivory Coast',
    'Syrian Arab Republic': 'Syria',
    'Yemen, Arab Republic': 'Yemen',
    'Congo': 'Congo Republic',
    'Dem. Rep. of the Congo': 'DR Congo'
}

df_ecom['Country'] = df_ecom['Country'].apply(lambda x: mapping.get(x, x))
df_ecom.head()

On effectue à nouveau la jointure.

In [ ]:
# join with the Country dataset on the common `Country` column
df_ecom2 = pd.merge(df_ecom, pays, on='Country', how='inner')
df_ecom2.shape

In [ ]:
# on vérie que tout colle
df_ecom['Country'].isin(df_ecom2['Country'].unique()).all()

On fusionne toutes les données nécessaires.

In [ ]:
# concatenate data
result2 = pd.concat([-pd.Series(result.T[0]),
                    -pd.Series(result.T[1]),
                    df_ecom2['Population'],
                    df_ecom2['Continent'],
                    df_ecom2['Country']],
                   axis=1)

result2 = result2.rename(columns={0:'x', 1:'y'})

# on trie par population décroissante afin que les grand cercles soient afficher en premier
result2 = result2.sort_values('Population', ascending=False)
result2.head()

### Amélioration de l'affichage

In [ ]:
# figure
fig, ax = plt.subplots(figsize=(12, 12))

# mapping des continents et des coleurs
c = result2['Continent'].map({'AF': 'r', 'EU': 'b', 'AS': 'y', 'NA': 'c', 'SA': 'm', 'OC': 'g'})

# scatterplot
sns.scatterplot(x='x',
                y='y',
                data=result2,
                hue=c,
                s=np.sqrt(result2['Population'])/10,  # formule trouvée par tatonnement
                legend=False,
                ax=ax)

# axes
ax.plot(plt.xlim(), [0.0, 0.0], ls=':', lw=1, color='k')
ax.plot([0.0, 0.0], plt.ylim(), ls=':', lw=1, color='k')

# labels
offset = 0.005
for label, x, y in zip(result2['Country'], result2['x'], result2['y']):
    plt.text(x+offset, y+offset, label, fontsize=8);

## Idem avec t-SNE (t-distributed Stochastic Neighbor Embedding)

T-distributed Stochastic Neighbor Embedding (t-SNE) un algorithme de machine learning pour la visualisation.

Il s'agit d'une technique non-linéaire de réduction de dimension pour visualiser des données multi-dimensionnelles dans un espace à 2 ou 3 dimensions. Particulièrement, l'algorithme représente chaque objet par un point dans un espace à 2 ou 3 dimensions de sorte que des objets similaires sont représentés par des points proches et des objets non similaires par des points éloignés avec une forte probabilité.

<div class="alert alert-success">
<b>Exercise</b>
<ul>
    <li>Importer la librairie <code>TSNE</code> de <code>sklearn.manifold</code></li>
    <li>Instancier un objet <code>TSNE</code> avec l'argument <code>perplexity=5</code> (qui représente le nombre de voisins proches utilisé par l'algorithme)</li>
    <li>Appliquer la méthode <code>fit_transform()</code> au dataset standardisé</li>
    <li>Ensuite afficher les résultats comme pour l'ACP</li>
</ul>
</div>